In [ ]:
import cv2
import random
import numpy as np
from PIL import Image
from skimage import color

from lib.lime import LIME
from lib.fog_gen import generate_fog
from lib.rain_gen import RainGenUsingNoise

In [ ]:
class RainEffectGenerator:
    def __init__(self):
        self._lime = LIME(iterations=25, alpha=1.0)
        self._illumination2darkness = {0: 1, 1: 0.75, 2: 0.65, 3: 0.5}
        self._weather2visibility = {'fog': (100,250), 'rain': (1000,2000), 'snow': (500, 1000)}
        self._illumination2fogcolor = {0: (80, 120), 1: (120, 160), 2: (160, 200), 3: (200, 240)}
        self._rain_layer_gen = RainGenUsingNoise()
        
    def getIlluminationMap(self, img: np.ndarray) -> np.ndarray: 
        self._lime.load(img)
        T = self._lime.illumMap()
        return T
    
    @staticmethod
    def reduce_lightHSV(rgb, sat_red=0.5, val_red=0.5):
        hsv = color.rgb2hsv(rgb/255)
        hsv[...,1] *= sat_red
        hsv[...,2] *= val_red
        return (color.hsv2rgb(hsv)*255).astype(np.uint8)
    
    @staticmethod
    def fogAttenuation(img: np.ndarray, depth:np.ndarray, visibility=1000, fog_color=200):
        img_fog = generate_fog(img.copy(), depth.copy(), visibility=visibility, fog_color=fog_color)
        return img_fog
    
    @staticmethod
    def illumination2opacity(img: np.ndarray, illumination_th):
        alpha = color.rgb2gray(img)
        alpha[alpha>illumination_th] = 1.0 - alpha[alpha>illumination_th]
        alpha_blur = cv2.blur(alpha, (11,11))
        return alpha_blur
        
    @staticmethod    
    def alphaBlend(img, layer, alpha):
        if layer.ndim==3:
            layer = cv2.cvtColor(layer.astype(np.uint8), cv2.COLOR_RGB2GRAY)
        blended = img*(1-alpha) + layer[:,:,None]*alpha
        return blended
    
    def genRainLayer(self):
        blur_angle = random.choice([-1, 1])*random.randint(60, 90)
        layer1 = self._rain_layer_gen.genRainLayer(h=720, 
                                                  w=1280, 
                                                  noise_scale=random.uniform(0.35, 0.55), 
                                                  noise_amount=0.5, 
                                                  zoom_layer=random.uniform(1.0, 3.5),
                                                  blur_kernel_size=random.choice([15, 17, 19, 21, 23]), 
                                                  blur_angle=blur_angle
                                                  )#large
        
        layer2 = self._rain_layer_gen.genRainLayer(h=720, 
                                                  w=1280, 
                                                  noise_scale=random.uniform(0.35, 0.55), 
                                                  noise_amount=0.15, 
                                                  zoom_layer=random.uniform(1.0, 3.5),
                                                  blur_kernel_size=random.choice([7, 9, 11, 13]), 
                                                  blur_angle=blur_angle
                                                  )#small
        # tr = random.random()*0.2+0.25
        layer = layer1 + layer2
        return layer
    
    def genEffect(self, img_path: str, depth_path: str, weather_type='rain'):
        I = np.array(Image.open(img_path))
        D = np.load(depth_path)
        
        T = self.getIlluminationMap(I)
        illumination_array = np.histogram(T, bins=4, range=(0,1))[0]/(T.size)
        illumination = illumination_array.argmax()
        
        if illumination>0:
            visibility = random.randint(self._weather2visibility[weather_type][0], self._weather2visibility[weather_type][1])
            fog_color = random.randint(self._illumination2fogcolor[illumination][0], self._illumination2fogcolor[illumination][1])
            I_dark = self.reduce_lightHSV(I, sat_red=self._illumination2darkness[illumination], val_red=self._illumination2darkness[illumination])
            I_fog = self.fogAttenuation(I_dark, D, visibility=visibility, fog_color=fog_color)
        else:
            fog_color = 75
            visibility = D.max()*0.75 if D.max()<1000 else 750
            I_fog = self.fogAttenuation(I, D, visibility=visibility, fog_color=fog_color)
        
        illm_th = illumination_array[illumination] if illumination<2 else 1-illumination_array[illumination]
        alpha_ch = self.illumination2opacity(I, illm_th)
        rain_layer = self.genRainLayer() 
        I_rain = self.alphaBlend(I_fog, rain_layer, alpha_ch[:,:,None])
        return I_rain.astype(np.uint8)

In [ ]:
raingen = RainEffectGenerator()

In [ ]:
save_folder = "../../dataset/train/rain_synthetic/images"
depth_folder = "rain_depth"

with open("rain_list.txt", "r") as f:
    content_files = f.read().splitlines() 
content_files = [Path(p) for p in content_files]

for path in tqdm(content_files):
    rainy = raingen.genEffect(path, os.path.join(depth_folder, path.stem+".npy"))
    Image.fromarray(rainy.astype(np.uint8)).save(os.path.join(save_folder, path.stem+"_rsyn.jpg"))

In [ ]:
img_name = "b1d7b3ac-36f2d3b7"
image_path = f"data/rgb/{img_name}.jpg"
depth_path = f"data/depth/{img_name}.npy"

In [ ]:
I_rain = we.genEffect(img_path=image_path, depth_path=depth_path, weather_type="rain")

In [ ]:
Image.fromarray(I_rain)

In [ ]:
Rain Effect:
    1. calculate illumination
    2. darkening image
    3. add fog
    4. calculate opacity
    5. gen rain layer
    6. blen rain layer to darkened image using alphablend function